# Correos electronicos SPAM: Un enfoque con Procesamiento de Lenguaje Natural

## Armando Misael Miranda Hernandez

Los correos electronicos no deseados en su bandeja de entreda son molestos ya que perturban la rutina del usuario. Es por eso que las cuentas de correo electronico ya tiene un filtro spam. Dado que es una de las aplicaciones del PLN mas utilizadas vamos a ver como se desarrollo un filtro de spam simple para correos electronicos.

In [2]:
# Importamos las famosas librerias
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [3]:
#instertamos los datos
full_corpus=pd.read_csv('SMSSpamCollection.tsv',sep='\t', header=None, names=['label','msg_body'])

#Separando los datos en ham y spam
ham_text=[]
spam_text=[]

## Bigrams

Los N-gramos se usan para modelar el lenguaje en funcion de la prediccion de palabras, es decir, predice la siguiente palabra de una oracion de palabras N-1 anteriores. Biagrams es la secuencia de 2 de N-gramos que predice la siguiente palabra de una oracion usando la palabra anterior. En lugar de considerar la historia completa de una oracion o una secuencia de palabras en particular, un modelo como biagram puede ser ocupado en terminos de una aproximacion de la historia al ocupar una historia limitada.

La identificacion de un mensaje como 'ham' o 'spam' es una tarea de clasificacion ya que la variable de destino tiene valores discretos que son "ham" o "spam". En esta practica, se usa el modelo biagram, aunque existen muchas tecnicas avanzadas que se pueden utilizar para este proposito. Para utilizar el modelo biagram para asignar un mensaje dado como "spam" o "ham", hay variospasos que deben lograrse:

#### 1. Inspeccion y separacion de mensajes en las categorias "Ham y Spam"
Inicialmente, el conjunto de datos debe inspeccionarse para ocuparlo y abordarlo para lograr la tarea. El formato de los datos dados, la cantidad de datos proporcionados la naturaleza de los datos se incluyen en esta inspeccion para indetificar la mejor aproximacion posible para la tarea.

El corpus de mensajes dado ha marcado cada mensaje como ham o spam. Ademas, hay 5568 mensajes en un DataFrame escrito en ingles que no son objetos nulos. Por lo tanto, el archivo tsv se puede leer usando DataFrame en python para clasificar esos mensajes de acuerdo con el indocador dado.

In [4]:
def separete_msg():
    for index, column in full_corpus.iterrows():
        label=column[0]
        message_text=column[1]
        if label=='ham':
            ham_text.append(message_text)
        if label=='spam':
            spam_text.append(message_text)
        
separete_msg()

### 2. Preprocesamiento de texto
El preprocesamiento es la tarea de realizar los pasos de preparacion e el corpus de texto sin formato para completar de manera eficiente una extraccion de texto o procesamiento de lenguaje natural o cualquier otra tarea que incluya texto sin formato. El preprocesamiento de texto consta de varios pasos, aunque algunos de ellos pueden no aplicarse a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto inlcuye los siguientes pasos de acuerdo con el conjunto de datos.

#### Eliminacion de signos de puntuacion

In [5]:
#Eliminacion de los signos de puntuacion de los mensajes de correo electronico
def remove_msg_punctuations(email_msg):
    puntuation_remove_msg="".join([word for Word in email_msg if word not in string.puntuaction])
    return puntuation_remove_msg

#### Convertir a minusculas:
Convertir a minusculas: La conversion de todos los caracteres del texto en un contexto comun, como lo soportes en minusculas, impide identificar dos palabras de manera diferente donde una esta en minisculas y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse como iguales, por lo tanto, poner en minusculas todos los caracteres facilita la tarea. Ademas, las palabras de detencion tambien estan en minusculas, por lo que esto tammbien haria posible eliminar palabras de detencion mas adelante.

#### Tokenizing
Tokenizing: La tokenizacion es la tarea de dividir el texto en partes significativas, es decir, tokens que incluye oraciones y palabras. Un token se puede considerar como una instancia de una secuancia de caracteres en un texto particular que se agrupan para proporcionar una unidad semantica util para su porterior procesamiento. En esta tarea, la tokenizacion de palabras se realiza combinando espacios en blanco entre palabras como delimitaor. Esto se logra en Python usando expresiones regulares para dividir una cadena en subcadenas con la funcion split(), que es un tokenizador basico.

In [6]:
# Convierte el texto en minusculas y tokenizing de palabras
def tokenize_into_words(text):
    tokens=re.split('\W', text)
    return tokens

#### Palabras Lematizantes:
La derivacion es el proceso de elimar afijos(sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raiz de palabra. Aunque la lematizacion esta relacionada con la derivacion, difiere ya que la lematizacion puede capturar formas canonicas basadas en el lema de una palabra. La lematizacion ocupa un vocabulario y un analisis morfologico de las palabras que lo hacen mas rapido y preciso que la derivacion. WordNetLemmatizer ha logrado la lematizacion en lenguaje Python

In [7]:
#Lemmatizing
word_lemmatizar=WordNetLemmatizer()
def lemmatization(tokenized_word):
    lemmatized_text=[word_lemmatizer.lematize(word)for word in tokenized_word]
    return ' '.join(lemmatized_text)

def preprocessing_msgs(corpus):
    categorized_text=pd.DataFrame(corpus)
    categorized_text['non_punc_message_body']=categorized_text[0].apply(lambda msg:remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body']=categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words']=categorized_text['tokenized_msg_body'].apply(lambda word_list:lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']

### 3. Extracción de características
Después de la etapa de procesamiento, las caracteristicas deben extraerse del texto. Las caracteristicas son las unidades que admiten la tarea de clasificación, y las bigrams son las caracteristicas en esta tarea de clasificación de mensajes. Los birams o las caracteristicas se extraen dek texto preprocesado Inicialmente, los unigramas se adquieren, y luego esos unigramas se utilizan para obtener los unigramas en cada corpus ("ham" y "spam").